In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import koreanize_matplotlib

import datetime
import random
import os
import sys


import sklearn
import catboost
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
def seed_everything(seed: int = 2024):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
seed_everything(2024)

In [3]:
train = pd.read_csv("./preprocessing_train.csv")
submission = pd.read_csv("./sample_submission.csv")

In [4]:
train.isnull().sum()

일시      0
최고기온    0
최저기온    0
일교차     0
강수량     0
평균습도    0
평균풍속    0
일조합     0
일사합     0
일조율     0
평균기온    0
dtype: int64

In [5]:
train["일시"] = pd.to_datetime(train["일시"])
submission["일시"] = pd.to_datetime(submission["일시"])

In [6]:
#년/월/일 추가
train['년']=train['일시'].dt.year
train['월']=train['일시'].dt.month
train['일']=train['일시'].dt.day

submission['년']=submission['일시'].dt.year
submission['월']=submission['일시'].dt.month
submission['일']=submission['일시'].dt.day

In [7]:
# 날짜 데이터
date_data = pd.date_range(start='1960-01-01', end='2022-12-31', freq='D')

# 일 단위 추출
day_of_year = date_data.dayofyear

# 주기 함수로 변환
train['Day sin'] = np.sin(2 * np.pi * day_of_year / 365)
train['Day cos'] = np.cos(2 * np.pi * day_of_year / 365)

# 날짜 데이터
date_data2 = pd.date_range(start='2023-01-01', end='2023-12-24', freq='D')

# 일 단위 추출
day_of_year2 = date_data2.dayofyear

# 주기 함수로 변환
submission['Day sin'] = np.sin(2 * np.pi * day_of_year2 / 358)
submission['Day cos'] = np.cos(2 * np.pi * day_of_year2 / 358)

In [8]:
# 주차 변수 추가
train["주차"] = train["일시"].map(lambda x: datetime.datetime(x.year, x.month, x.day).isocalendar()[1])
submission["주차"] = submission["일시"].map(lambda x: datetime.datetime(x.year, x.month, x.day).isocalendar()[1])

In [9]:
# 계절 변수 생성
train['계절'] = ''

for i, month in enumerate(train['월']):
    if 3 <= month <= 5:
        train.at[i, '계절'] = '봄'
    elif 6 <= month <= 8:
        train.at[i, '계절'] = '여름'
    elif 9 <= month <= 11:
        train.at[i, '계절'] = '가을'
    else:
        train.at[i, '계절'] = '겨울'

# 결과 확인
print(train[['일시', '월', '계절']])


# 계절 변수 생성
submission['계절'] = ''

for i, month in enumerate(submission['월']):
    if 3 <= month <= 5:
        submission.at[i, '계절'] = '봄'
    elif 6 <= month <= 8:
        submission.at[i, '계절'] = '여름'
    elif 9 <= month <= 11:
        submission.at[i, '계절'] = '가을'
    else:
        submission.at[i, '계절'] = '겨울'

# 결과 확인
print(submission[['일시', '월', '계절']])

              일시   월  계절
0     1960-01-01   1  겨울
1     1960-01-02   1  겨울
2     1960-01-03   1  겨울
3     1960-01-04   1  겨울
4     1960-01-05   1  겨울
...          ...  ..  ..
23006 2022-12-27  12  겨울
23007 2022-12-28  12  겨울
23008 2022-12-29  12  겨울
23009 2022-12-30  12  겨울
23010 2022-12-31  12  겨울

[23011 rows x 3 columns]
            일시   월  계절
0   2023-01-01   1  겨울
1   2023-01-02   1  겨울
2   2023-01-03   1  겨울
3   2023-01-04   1  겨울
4   2023-01-05   1  겨울
..         ...  ..  ..
353 2023-12-20  12  겨울
354 2023-12-21  12  겨울
355 2023-12-22  12  겨울
356 2023-12-23  12  겨울
357 2023-12-24  12  겨울

[358 rows x 3 columns]


In [10]:
# '계절' 열에 대한 원핫인코딩 수행
one_hot_encoded = pd.get_dummies(train['계절'], prefix='계절')

# 기존 데이터프레임과 합치기
train = pd.concat([train, one_hot_encoded], axis=1)

# '계절' 열과 중복된 열 삭제
train.drop(['계절'], axis=1, inplace=True)

# 결과 확인
print(train)


# '계절' 열에 대한 원핫인코딩 수행
one_hot_encoded = pd.get_dummies(submission['계절'], prefix='계절')

# 기존 데이터프레임과 합치기
submission = pd.concat([submission, one_hot_encoded], axis=1)

# '계절' 열과 중복된 열 삭제
submission.drop(['계절'], axis=1, inplace=True)

# 결과 확인
print(train)
print(submission)

              일시  최고기온  최저기온   일교차  강수량  평균습도  평균풍속  일조합        일사합  \
0     1960-01-01   2.2  -5.2   7.4  0.4  68.3   1.7  6.7  13.038944   
1     1960-01-02   1.2  -5.6   6.8  0.4  87.7   1.3  0.0   4.181680   
2     1960-01-03   8.7  -2.1  10.8  0.0  81.3   3.0  0.0   4.181680   
3     1960-01-04  10.8   1.2   9.6  0.0  79.7   4.4  2.6   7.618827   
4     1960-01-05   1.3  -8.2   9.5  0.0  44.0   5.1  8.2  15.021913   
...          ...   ...   ...   ...  ...   ...   ...  ...        ...   
23006 2022-12-27   3.3  -7.3  10.6  0.1  69.8   1.8  8.8  10.250000   
23007 2022-12-28   0.1  -6.0   6.1  0.1  58.1   2.5  8.7  10.860000   
23008 2022-12-29   2.1  -7.8   9.9  0.0  56.3   1.7  9.0  10.880000   
23009 2022-12-30   2.3  -4.4   6.7  0.0  65.6   1.9  7.9  10.840000   
23010 2022-12-31   2.1  -5.1   7.2  0.0  65.5   1.4  1.1   4.160000   

             일조율  ...     년   월   일       Day sin   Day cos  주차  계절_가을  계절_겨울  \
0      55.602262  ...  1960   1   1  1.721336e-02  0.999852  53  F

In [11]:
train

,일시,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,...,년,월,일,Day sin,Day cos,주차,계절_가을,계절_겨울,계절_봄,계절_여름
0,1960-01-01,2.2,-5.2,7.4,0.4,68.3,1.7,6.7,13.038944,55.602262,...,1960,1,1,1.721336e-02,0.999852,53,False,True,False,False
1,1960-01-02,1.2,-5.6,6.8,0.4,87.7,1.3,0.0,4.181680,2.124736,...,1960,1,2,3.442161e-02,0.999407,53,False,True,False,False
2,1960-01-03,8.7,-2.1,10.8,0.0,81.3,3.0,0.0,4.181680,2.124736,...,1960,1,3,5.161967e-02,0.998667,53,False,True,False,False
3,1960-01-04,10.8,1.2,9.6,0.0,79.7,4.4,2.6,7.618827,22.877209,...,1960,1,4,6.880243e-02,0.997630,1,False,True,False,False
4,1960-01-05,1.3,-8.2,9.5,0.0,44.0,5.1,8.2,15.021913,67.574842,...,1960,1,5,8.596480e-02,0.996298,1,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23006,2022-12-27,3.3,-7.3,10.6,0.1,69.8,1.8,8.8,10.250000,91.700000,...,2022,12,27,-6.880243e-02,0.997630,52,False,True,False,False
23007,2022-12-28,0.1,-6.0,6.1,0.1,58.1,2.5,8.7,10.860000,90.600000,...,2022,12,28,-5.161967e-02,0.998667,52,False,True,False,False
23008,2022-12-29,2.1,-7.8,9.9,0.0,56.3,1.7,9.0,10.880000,93.800000,...,2022,12,29,-3.442161e-02,0.999407,52,False,True,False,False
23009,2022-12-30,2.3,-4.4,6.7,0.0,65.6,1.9,7.9,10.840000,82.300000,...,2022,12,30,-1.721336e-02,0.999852,52,False,True,False,False


In [12]:
submission

,일시,평균기온,년,월,일,Day sin,Day cos,주차,계절_가을,계절_겨울,계절_봄,계절_여름
0,2023-01-01,0,2023,1,1,1.754990e-02,0.999846,52,False,True,False,False
1,2023-01-02,0,2023,1,2,3.509439e-02,0.999384,1,False,True,False,False
2,2023-01-03,0,2023,1,3,5.262807e-02,0.998614,1,False,True,False,False
3,2023-01-04,0,2023,1,4,7.014554e-02,0.997537,1,False,True,False,False
4,2023-01-05,0,2023,1,5,8.764140e-02,0.996152,1,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
353,2023-12-20,0,2023,12,20,-7.014554e-02,0.997537,51,False,True,False,False
354,2023-12-21,0,2023,12,21,-5.262807e-02,0.998614,51,False,True,False,False
355,2023-12-22,0,2023,12,22,-3.509439e-02,0.999384,51,False,True,False,False
356,2023-12-23,0,2023,12,23,-1.754990e-02,0.999846,51,False,True,False,False


일교차 예측

In [13]:
# 필요한 변수 선택 (예시로 선택된 변수들이므로 실제로 사용하는 변수에 맞게 수정 필요)
selected_features = ['년', '월', '일', 'Day sin', 'Day cos', '주차','계절_가을','계절_겨울','계절_봄','계절_여름']
target_variable = '일교차'

# 선택된 변수들을 사용하여 훈련 데이터셋 준비
train_x = train[selected_features]
train_y = train[target_variable]

test_x = submission[selected_features]

# 모델 정의 (verbose 값을 metric_period의 배수로 설정)
cat = CatBoostRegressor(random_state=2024,
                        n_estimators=1000,
                        learning_rate=0.01,
                        depth=10,
                        l2_leaf_reg=3,
                        metric_period=1000,
                        verbose=1000)  # verbose를 metric_period의 배수로 설정

# 모델 학습
cat.fit(train_x, train_y)

0:	learn: 2.9026863	total: 156ms	remaining: 2m 36s
999:	learn: 2.5650494	total: 10.3s	remaining: 0us


In [14]:
# submission 데이터에서 필요한 변수만 남기기 (평균기온을 예측하기 위한 변수)
submission_data = submission[['년', '월', '일', 'Day sin', 'Day cos', '주차','계절_가을','계절_겨울','계절_봄','계절_여름']]

# 예측 수행
submission_pred = cat.predict(submission_data)

In [15]:
submission['일교차'] = submission_pred

In [16]:
submission

,일시,평균기온,년,월,일,Day sin,Day cos,주차,계절_가을,계절_겨울,계절_봄,계절_여름,일교차
0,2023-01-01,0,2023,1,1,1.754990e-02,0.999846,52,False,True,False,False,9.146866
1,2023-01-02,0,2023,1,2,3.509439e-02,0.999384,1,False,True,False,False,7.932401
2,2023-01-03,0,2023,1,3,5.262807e-02,0.998614,1,False,True,False,False,8.341053
3,2023-01-04,0,2023,1,4,7.014554e-02,0.997537,1,False,True,False,False,8.391090
4,2023-01-05,0,2023,1,5,8.764140e-02,0.996152,1,False,True,False,False,8.440500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,2023-12-20,0,2023,12,20,-7.014554e-02,0.997537,51,False,True,False,False,8.697825
354,2023-12-21,0,2023,12,21,-5.262807e-02,0.998614,51,False,True,False,False,8.639318
355,2023-12-22,0,2023,12,22,-3.509439e-02,0.999384,51,False,True,False,False,8.609824
356,2023-12-23,0,2023,12,23,-1.754990e-02,0.999846,51,False,True,False,False,8.495345


일죠율 예측

In [17]:
# 필요한 변수 선택 (예시로 선택된 변수들이므로 실제로 사용하는 변수에 맞게 수정 필요)
selected_features = ['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차','계절_가을','계절_겨울','계절_봄','계절_여름']
target_variable = '일조율'

# 선택된 변수들을 사용하여 훈련 데이터셋 준비
train_x = train[selected_features]
train_y = train[target_variable]

test_x = submission[selected_features]

# 모델 정의 (verbose 값을 metric_period의 배수로 설정)
cat = CatBoostRegressor(random_state=2024,
                        n_estimators=1000,
                        learning_rate=0.01,
                        depth=10,
                        l2_leaf_reg=3,
                        metric_period=1000,
                        verbose=1000)  # verbose를 metric_period의 배수로 설정

# 모델 학습
cat.fit(train_x, train_y)

0:	learn: 31.5021425	total: 11.5ms	remaining: 11.5s
999:	learn: 21.1728511	total: 10.7s	remaining: 0us


In [18]:
# submission 데이터에서 필요한 변수만 남기기 (평균기온을 예측하기 위한 변수)
submission_data = submission[['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차','계절_가을','계절_겨울','계절_봄','계절_여름']]

# 예측 수행
submission_pred = cat.predict(submission_data)

In [19]:
submission['일조율'] = submission_pred

평균습도 예측

In [20]:
# 필요한 변수 선택 (예시로 선택된 변수들이므로 실제로 사용하는 변수에 맞게 수정 필요)
selected_features = ['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차', '일조율','계절_가을','계절_겨울','계절_봄','계절_여름']
target_variable = '평균습도'

# 선택된 변수들을 사용하여 훈련 데이터셋 준비
train_x = train[selected_features]
train_y = train[target_variable]

test_x = submission[selected_features]

# 모델 정의 (verbose 값을 metric_period의 배수로 설정)
cat = CatBoostRegressor(random_state=2024,
                        n_estimators=1000,
                        learning_rate=0.01,
                        depth=10,
                        l2_leaf_reg=3,
                        metric_period=1000,
                        verbose=1000)  # verbose를 metric_period의 배수로 설정

# 모델 학습
cat.fit(train_x, train_y)

0:	learn: 14.4696910	total: 7.72ms	remaining: 7.71s
999:	learn: 8.4144786	total: 11.7s	remaining: 0us


In [21]:
# submission 데이터에서 필요한 변수만 남기기 (평균기온을 예측하기 위한 변수)
submission_data = submission[['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차', '일조율','계절_가을','계절_겨울','계절_봄','계절_여름']]

# 예측 수행
submission_pred = cat.predict(submission_data)

In [22]:
submission['평균습도'] = submission_pred

강수량 예측

In [23]:
# 필요한 변수 선택 (예시로 선택된 변수들이므로 실제로 사용하는 변수에 맞게 수정 필요)
selected_features = ['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차', '일조율', '평균습도','계절_가을','계절_겨울','계절_봄','계절_여름']
target_variable = '강수량'

# 선택된 변수들을 사용하여 훈련 데이터셋 준비
train_x = train[selected_features]
train_y = train[target_variable]

test_x = submission[selected_features]

# 모델 정의 (verbose 값을 metric_period의 배수로 설정)
cat = CatBoostRegressor(random_state=2024,
                        n_estimators=1000,
                        learning_rate=0.01,
                        depth=10,
                        l2_leaf_reg=3,
                        metric_period=1000,
                        verbose=1000)  # verbose를 metric_period의 배수로 설정

# 모델 학습
cat.fit(train_x, train_y)

0:	learn: 17.2185045	total: 14.2ms	remaining: 14.2s
999:	learn: 12.3011655	total: 12.7s	remaining: 0us


In [24]:
# submission 데이터에서 필요한 변수만 남기기 (평균기온을 예측하기 위한 변수)
submission_data = submission[['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차', '일조율', '평균습도','계절_가을','계절_겨울','계절_봄','계절_여름']]

# 예측 수행
submission_pred = cat.predict(submission_data)

In [25]:
submission['강수량'] = submission_pred

In [26]:
submission

,일시,평균기온,년,월,일,Day sin,Day cos,주차,계절_가을,계절_겨울,계절_봄,계절_여름,일교차,일조율,평균습도,강수량
0,2023-01-01,0,2023,1,1,1.754990e-02,0.999846,52,False,True,False,False,9.146866,57.351513,60.075469,0.800845
1,2023-01-02,0,2023,1,2,3.509439e-02,0.999384,1,False,True,False,False,7.932401,69.278996,56.842441,-0.094986
2,2023-01-03,0,2023,1,3,5.262807e-02,0.998614,1,False,True,False,False,8.341053,66.702954,57.158374,-0.022009
3,2023-01-04,0,2023,1,4,7.014554e-02,0.997537,1,False,True,False,False,8.391090,65.459941,56.892470,0.075621
4,2023-01-05,0,2023,1,5,8.764140e-02,0.996152,1,False,True,False,False,8.440500,64.765867,56.681623,0.054950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,2023-12-20,0,2023,12,20,-7.014554e-02,0.997537,51,False,True,False,False,8.697825,65.536562,63.345978,1.257310
354,2023-12-21,0,2023,12,21,-5.262807e-02,0.998614,51,False,True,False,False,8.639318,65.091690,63.488225,1.238024
355,2023-12-22,0,2023,12,22,-3.509439e-02,0.999384,51,False,True,False,False,8.609824,65.235299,63.532520,1.031208
356,2023-12-23,0,2023,12,23,-1.754990e-02,0.999846,51,False,True,False,False,8.495345,65.178277,63.585894,0.828671


평균기온 예측

In [35]:
# 필요한 변수 선택 (예시로 선택된 변수들이므로 실제로 사용하는 변수에 맞게 수정 필요)
selected_features = ['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차', '일조율', '평균습도', '강수량','계절_가을','계절_겨울','계절_봄','계절_여름']
target_variable = '평균기온'

# 선택된 변수들을 사용하여 훈련 데이터셋 준비
train_x = train[selected_features]
train_y = train[target_variable]

test_x = submission[selected_features]

# RandomForestRegressor 모델 정의
rf_model = RandomForestRegressor(random_state=2024, n_estimators=100)

# CatBoostRegressor 모델 정의
cat_model = CatBoostRegressor(random_state=2024, n_estimators=1000, learning_rate=0.01, depth=10, l2_leaf_reg=3)

# VotingRegressor 정의 (voting='soft'로 설정)
voting_regressor = VotingRegressor(estimators=[
    ('rf', rf_model),
    ('catboost', cat_model)
], weights = [1, 3])

# 모델 학습
voting_regressor.fit(train_x, train_y)

KeyError: "None of [Index(['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차', '일조율', '평균습도', '강수량',\n       '계절_가을', '계절_겨울', '계절_봄', '계절_여름'],\n      dtype='object')] are in the [columns]"

In [29]:
# submission 데이터에서 필요한 변수만 남기기 (평균기온을 예측하기 위한 변수)
submission_data = submission[['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차', '일조율', '평균습도', '강수량','계절_가을','계절_겨울','계절_봄','계절_여름']]

# 예측 수행
submission_pred = voting_regressor.predict(submission_data)

In [30]:
submission = pd.read_csv('./sample_submission.csv')
submission['평균기온']=submission_pred

In [31]:
submission

,일시,평균기온
0,2023-01-01,-1.298566
1,2023-01-02,-1.734372
2,2023-01-03,-1.802575
3,2023-01-04,-1.596439
4,2023-01-05,-1.604680
...,...,...
353,2023-12-20,-1.032324
354,2023-12-21,-0.994895
355,2023-12-22,-1.099842
356,2023-12-23,-1.249417


In [32]:
submission.to_csv('votingregressor2.csv', index=False)